In [ ]:
def hrf_convo(design_matrix,bf):
    """
    Takes designmatrix upsampled to TR/TReso and convolves it with canonical HRF
    Args
        design_matrix: C * VolTreso ndarray; indicates closest Tr/TReso timepoint where an event started for specific regressor
        bf: basis function - will usually be the hrf; 1D array
    Returns
        design_matrix_conv: C*Vol array with predicted BOLD response
    """
    design_matrix_pad=np.pad(design_matrix,((0,len(bf)),(0, 0))) # pad with zeros, so we can fit the hrf also for very late events
    design_matrix_conv=np.empty(design_matrix_pad.shape) # prepare convolved design matrix
    for C in range(len(design_matrix[1])): # for each condition C

        design_matrix_conv[:,C]=np.convolve(design_matrix_pad[:len(design_matrix_pad[0])-(len(bf)+1),C],bf) #convolute with design with bf

    design_matrix_downsampled=design_matrix_conv[:len(design_matrix_pad[0])-(len(bf)+2),:] # cut back the padded stuff

    design_matrix_downsampled=design_matrix_downsampled[0::10,:] # pick every time-point where we have a volume
    constant=np.ones((len(design_matrix_downsampled),1),dtype=int) # generate constant (intercept)
    design_matrix_downsampled=np.append(design_matrix_downsampled,constant,axis=1) #append intercept to design matrix
    return design_matrix_downsampled